This notebook is not used operationally or for any validation, its only purpose is to have a clear understanding of the core functions of the AA workflow. The outputs and dimensions of each main step can thus be identified here.

**Import required libraries and functions**

In [2]:
%cd ..

c:\Users\amine.barkaoui\OneDrive - World Food Programme\Documents\GitHub\anticipatory-action


In [4]:
import os
import datetime
import pandas as pd

from config.params import Params

from AA.helper_fns import (
    read_forecasts,
    read_observations,
    aggregate_by_district,
    merge_un_biased_probs,
    merge_probabilities_triggers_dashboard,
)

from hip.analysis.analyses.drought import (
    get_accumulation_periods,
    run_accumulation_index,
    run_gamma_standardization,
    run_bias_correction,
    compute_probabilities,
)

from hip.analysis.aoi.analysis_area import AnalysisArea

**Define parameters**

The `config/{country}_config.yaml` file gathers all the parameters used in the operational script and that can be customized. For example, the *monitoring_year*, the list of districts or the intensity levels can be defined in that file.

In [35]:
params = Params(iso='ZWE', issue=10, index='SPI')
params.monitoring_year = 2023

**Read shapefile**

In [6]:
# Define aoi to read datasets using hip-analysis
area = AnalysisArea.from_admin_boundaries(
    iso3=params.iso.upper(),
    admin_level=2,
    resolution=0.25,
    datetime_range=f"1981-01-01/{params.monitoring_year + 1}-06-30",
)

# Read the shapefile
gdf = area.get_dataset([area.BASE_AREA_DATASET])
gdf

,geometry,Code,Name,adm1_Code,adm0_Code
Name,,,,,
Bulawayo,"POLYGON ((28.6712 -20.0163, 28.6072 -19.9615, ...",1010745,Bulawayo,900969,271
Chitungwiza,"POLYGON ((31 -17.9994, 31.0626 -18.0506, 31.11...",1010758,Chitungwiza,900970,271
Epworth,"POLYGON ((31.2031 -17.8863, 31.1303 -17.8654, ...",1010760,Epworth,900970,271
Harare,"POLYGON ((31.2031 -17.8863, 31.2195 -17.8512, ...",1010771,Harare,900970,271
Harare Rural,"POLYGON ((31.1361 -17.9289, 31.125 -17.888, 31...",1010772,Harare Rural,900970,271
...,...,...,...,...,...
Redcliff,"POLYGON ((29.8419 -19.025, 29.8195 -19.0142, 2...",1010810,Redcliff,900978,271
Shurugwi,"POLYGON ((30.4711 -19.8304, 30.4674 -19.8254, ...",1010817,Shurugwi,900978,271
Shurugwi Town,"POLYGON ((30.0317 -19.6013, 29.9984 -19.607, 3...",1010818,Shurugwi Town,900978,271


**Read forecasts**

In [46]:
# When update is set to False, the downscaled dataset is read from a local folder or a s3 bucket. Otherwise, it is directly read from HDC.
forecasts = read_forecasts(
    area,
    params.issue,
    f"{params.data_path}/data/{params.iso}/zarr/2022/{str(params.issue).zfill(2)}/forecasts.zarr",
    update=False,  # True,
)
forecasts

<xarray.DataArray 'tp' (time: 9127, ensemble: 51, latitude: 28, longitude: 33)> Size: 2GB
dask.array<open_dataset-tp, shape=(9127, 51, 28, 33), dtype=float32, chunksize=(9127, 1, 28, 33), chunktype=numpy.ndarray>
Coordinates:
  * ensemble     (ensemble) int64 408B 0 1 2 3 4 5 6 7 ... 44 45 46 47 48 49 50
    issue        (time) <U7 256kB dask.array<chunksize=(9127,), meta=np.ndarray>
  * latitude     (latitude) float64 224B -15.62 -15.88 -16.12 ... -22.12 -22.38
  * longitude    (longitude) float64 264B 25.12 25.38 25.62 ... 32.88 33.12
    spatial_ref  int32 4B ...
  * time         (time) datetime64[ns] 73kB 1981-10-01 1981-10-02 ... 2024-04-30
Attributes:
    nodata:   nan
    units:    mm

**Read observations**

In [47]:
# Observations data reading (already stored as the dataset used is the same as the one used in the pre-season/analytical script)
observations = read_observations(
    area,
    f"{params.data_path}/data/{params.iso}/zarr/{params.calibration_year}/obs/observations.zarr",
)
observations

<xarray.DataArray 'band' (time: 15156, latitude: 28, longitude: 33)> Size: 112MB
dask.array<open_dataset-band, shape=(15156, 28, 33), dtype=float64, chunksize=(1, 28, 33), chunktype=numpy.ndarray>
Coordinates:
  * latitude     (latitude) float64 224B -15.62 -15.88 -16.12 ... -22.12 -22.38
  * longitude    (longitude) float64 264B 25.12 25.38 25.62 ... 32.88 33.12
    spatial_ref  int32 4B ...
  * time         (time) datetime64[ns] 121kB 1981-01-01 ... 2022-06-30
Attributes:
    nodata:   nan

**Read pre-computed triggers**

Now that we got all the data we need, let's read the triggers file so we can merge the probabilities with it once we have them.

In [27]:
# Read triggers file
if os.path.exists(f"{params.data_path}/data/{params.iso}/probs/aa_probabilities_triggers_pilots.csv"):
    triggers_df = pd.read_csv(
        f"{params.data_path}/data/{params.iso}/probs/aa_probabilities_triggers_pilots.csv",
    )
else:
    triggers_df = pd.read_csv(
        f"{params.data_path}/data/{params.iso}/triggers/triggers.spi.dryspell.{params.calibration_year}.pilots.csv",
    )
triggers_df

,district,index,category,window,issue_ready,issue_set,trigger_ready,trigger_set,vulnerability,prob_ready,prob_set
0,Beitbridge,DRYSPELL JF,Normal,Window 2,7.0,8.0,0.14,0.30,NRT,NaN,NaN
1,Beitbridge,SPI DJF,Normal,Window 2,7.0,8.0,0.38,0.00,NRT,NaN,NaN
2,Beitbridge,SPI FM,Normal,Window 2,9.0,10.0,0.31,0.35,NRT,NaN,NaN
3,Beitbridge,SPI JFM,Normal,Window 2,10.0,11.0,0.00,0.37,NRT,NaN,NaN
4,Beitbridge,SPI ND,Normal,Window 1,6.0,7.0,0.35,0.11,NRT,0.24,NaN
...,...,...,...,...,...,...,...,...,...,...,...
73,Rushinga,SPI DJ,Normal,Window 2,9.0,10.0,0.23,0.37,NRT,NaN,NaN
74,Rushinga,SPI FM,Normal,Window 2,11.0,12.0,0.34,0.27,NRT,NaN,NaN
75,Rushinga,SPI JF,Normal,Window 2,8.0,9.0,0.08,0.34,NRT,NaN,NaN
76,Rushinga,SPI ND,Normal,Window 1,6.0,7.0,0.30,0.26,NRT,0.29,NaN


**Get accumulation periods covered by the forecasts of the defined issue month**

In [15]:
# Get accumulation periods (DJ, JF, FM, DJF, JFM...)
accumulation_periods = get_accumulation_periods(
    forecasts,
    params.start_season,
    params.end_season,
    params.min_index_period,
    params.max_index_period,
)
accumulation_periods

{'JF': (1, 2),
 'FM': (2, 3),
 'MA': (3, 4),
 'ON': (10, 11),
 'ND': (11, 12),
 'DJ': (12, 1),
 'JFM': (1, 2, 3),
 'FMA': (2, 3, 4),
 'OND': (10, 11, 12),
 'NDJ': (11, 12, 1),
 'DJF': (12, 1, 2)}

Here we focus on the pipeline for one indicator (one period) so we select a single element from the above dictionary (November-December using October forecasts).

In [16]:
# Get single use case
period_name, period_months = list(accumulation_periods.items())[4]
period_name, period_months

('ND', (11, 12))

**Run accumulation (sum for SPI)**

In [17]:
# Remove 1980 season to harmonize observations between different indexes 
if int(params.issue) >= params.start_monitoring:
    observations = observations.where(
        observations.time.dt.date >= datetime.date(1981, 10, 1), drop=True
    )

In [23]:
# Accumulation
accumulation_fc = run_accumulation_index(
    forecasts.chunk(dict(time=-1)), params.aggregate, period_months, forecasts=True
)
accumulation_obs = run_accumulation_index(
    observations.chunk(dict(time=-1)), params.aggregate, period_months
)

In [33]:
accumulation_fc

<xarray.DataArray 'tp' (time: 43, ensemble: 51, latitude: 28, longitude: 33)> Size: 8MB
array([[[[269.66193 , 268.3058  , 272.03986 , ..., 242.80408 ,
          257.6632  , 272.5225  ],
         [272.76898 , 272.01178 , 275.52374 , ..., 247.98917 ,
          257.74625 , 267.50323 ],
         [275.87607 , 275.71774 , 279.0074  , ..., 253.17432 ,
          257.8292  , 262.48413 ],
         ...,
         [199.7018  , 201.42699 , 202.37112 , ..., 285.85434 ,
          272.5029  , 259.15146 ],
         [189.30959 , 188.2329  , 188.99011 , ..., 303.66135 ,
          290.905   , 278.14865 ],
         [178.9174  , 175.0388  , 175.6091  , ..., 321.46835 ,
          309.3071  , 297.14584 ]],

        [[359.0663  , 352.1869  , 352.1893  , ..., 263.9928  ,
          255.48695 , 246.98116 ],
         [348.61826 , 341.00363 , 340.82388 , ..., 247.46465 ,
          238.9989  , 230.53311 ],
         [338.17035 , 329.82043 , 329.45847 , ..., 230.93654 ,
          222.5108  , 214.08511 ],
...
         [171.82654 , 161.47784 , 156.27957 , ..., 198.6933  ,
          194.31589 , 189.93857 ],
         [165.49377 , 156.16245 , 151.49783 , ..., 185.14917 ,
          188.10751 , 191.06577 ],
         [159.1608  , 150.84726 , 146.71613 , ..., 171.60513 ,
          181.89905 , 192.19305 ]],

        [[460.15686 , 461.8687  , 461.0348  , ..., 297.03302 ,
          290.4294  , 283.8256  ],
         [444.64917 , 444.1084  , 442.51062 , ..., 279.74442 ,
          273.3686  , 266.99283 ],
         [429.14154 , 426.34827 , 423.98645 , ..., 262.4557  ,
          256.30786 , 250.15997 ],
         ...,
         [221.34328 , 225.47127 , 229.79639 , ..., 298.45908 ,
          299.73886 , 301.0187  ],
         [213.1679  , 215.04045 , 218.76624 , ..., 305.58682 ,
          309.67044 , 313.75406 ],
         [204.99254 , 204.6096  , 207.73602 , ..., 312.71457 ,
          319.60202 , 326.48947 ]]]], dtype=float32)
Coordinates:
  * ensemble     (ensemble) int64 408B 0 1 2 3 4 5 6 7 ... 44 45 46 47 48 49 50
  * latitude     (latitude) float64 224B -15.62 -15.88 -16.12 ... -22.12 -22.38
  * longitude    (longitude) float64 264B 25.12 25.38 25.62 ... 32.88 33.12
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 344B 1981-01-01 1982-01-01 ... 2023-01-01
Attributes:
    nodata:   nan
    units:    mm

In [25]:
accumulation_obs

<xarray.DataArray 'band' (time: 41, latitude: 28, longitude: 33)> Size: 303kB
dask.array<concatenate, shape=(41, 28, 33), dtype=float64, chunksize=(1, 28, 33), chunktype=numpy.ndarray>
Coordinates:
  * latitude     (latitude) float64 224B -15.62 -15.88 -16.12 ... -22.12 -22.38
  * longitude    (longitude) float64 264B 25.12 25.38 25.62 ... 32.88 33.12
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 328B 1981-01-01 1982-01-01 ... 2021-01-01
Attributes:
    nodata:   nan

**Run standardization (SPI)**

In [19]:
# Remove inconsistent observations
accumulation_obs = accumulation_obs.sel(
    time=slice(datetime.date(1979, 1, 1), datetime.date(params.monitoring_year - 1, 12, 31))
)

In [28]:
# Anomaly
anomaly_fc = run_gamma_standardization(
    accumulation_fc.load(),
    params.hist_anomaly_start,
    params.hist_anomaly_stop,
    members=True,
)
anomaly_obs = run_gamma_standardization(
    accumulation_obs.load(),
    params.hist_anomaly_start,
    params.hist_anomaly_stop,
)

c:\Users\amine.barkaoui\AppData\Local\miniconda3\envs\aa-env\lib\site-packages\hip\analysis\analyses\drought.py:232: FutureWarning: updating coordinate 'time' with a PandasMultiIndex would leave the multi-index level coordinates ['date', 'ensemble'] in an inconsistent state. This will raise an error in the future. Use `.drop_vars(['time', 'date', 'ensemble'])` before assigning new coordinate values.
  rfh["time"] = [


In [31]:
anomaly_fc

<xarray.DataArray 'tp' (latitude: 28, longitude: 33, time: 1257)> Size: 9MB
array([[[-865.,  202.,   53., ...,  107., 1270., 1213.],
        [-842.,  145.,   21., ...,  172., 1478., 1222.],
        [-787.,  138.,  -47., ...,  243., 1587., 1192.],
        ...,
        [-414., -188., -297., ..., 1087.,  326.,  140.],
        [-303., -326., -382., ...,  926.,  231.,   32.],
        [-191., -463., -463., ...,  753.,  134.,  -77.]],

       [[-697.,  203.,   77., ...,   -3., 1282., 1172.],
        [-660.,  151.,   51., ...,   86., 1451., 1181.],
        [-607.,  149.,   13., ...,  168., 1537., 1153.],
        ...,
        [-210., -216., -383., ..., 1089.,  218.,  120.],
        [-131., -336., -454., ...,  945.,  137.,   33.],
        [ -50., -456., -522., ...,  790.,   55.,  -56.]],

       [[-524.,  204.,  103., ..., -115., 1284., 1123.],
        [-471.,  158.,   83., ...,   -3., 1410., 1129.],
        [-420.,  160.,   77., ...,   91., 1471., 1104.],
        ...,
...
        ...,
        [ 850., -439., -171., ...,  476.,  -51.,  965.],
        [ 708., -637., -151., ...,  397., -131.,  962.],
        [ 555., -836., -126., ...,  315., -207.,  952.]],

       [[ 481.,   72., 1000., ..., -710.,  126.,  809.],
        [ 464.,  181., 1025., ..., -692.,  -28.,  835.],
        [ 454.,  256., 1024., ..., -756., -128.,  864.],
        ...,
        [1058., -171., -235., ...,  232., -153., 1075.],
        [ 934., -374., -265., ...,  170., -146., 1102.],
        [ 796., -583., -290., ...,  108., -134., 1121.]],

       [[ 382.,   44., 1045., ..., -522.,   76.,  753.],
        [ 321.,  142., 1074., ..., -510.,  -66.,  748.],
        [ 305.,  224., 1062., ..., -591., -163.,  769.],
        ...,
        [1244.,   83., -294., ...,  -28., -254., 1171.],
        [1141., -120., -379., ...,  -72., -157., 1229.],
        [1020., -333., -459., ..., -113.,  -57., 1275.]]])
Coordinates:
  * latitude     (latitude) float64 224B -15.62 -15.88 -16.12 ... -22.12 -22.38
  * longitude    (longitude) float64 264B 25.12 25.38 25.62 ... 32.88 33.12
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 10kB 1981-01-01 1981-01-02 ... 2023-02-20
Attributes:
    nodata:                 nan
    units:                  mm
    spi_calibration_begin:  1981-01-01 00:00:00
    spi_calibration_end:    2018-02-20 00:00:00

In [32]:
anomaly_obs

<xarray.DataArray 'band' (latitude: 28, longitude: 33, time: 41)> Size: 303kB
array([[[-2552.,  -196.,   663., ..., -2228.,   764., -1598.],
        [-2511.,  -513.,   714., ..., -2306.,  1055., -1437.],
        [-2358.,  -699.,   676., ..., -2374.,  1200., -1426.],
        ...,
        [ -745.,  -775.,  -911., ...,  -773.,   785., -2571.],
        [ -835., -1295.,  -706., ...,  -643.,  1139., -2595.],
        [ -977., -1464.,  -714., ...,  -479.,  1278., -2177.]],

       [[-2106.,  -446.,   509., ..., -2528.,   834., -1484.],
        [-1936.,  -947.,   558., ..., -2553.,   973., -1670.],
        [-1729., -1062.,   639., ..., -2546.,  1012., -1630.],
        ...,
        [ -723.,  -599., -1108., ...,  -943.,   746., -2721.],
        [ -726., -1178., -1038., ...,  -833.,   861., -2770.],
        [ -913., -1263.,  -852., ...,  -680.,  1107., -2439.]],

       [[-1689.,  -706.,   914., ..., -2701.,   874., -1319.],
        [-1489., -1072.,   877., ..., -2592.,   854., -1580.],
        [-1457., -1155.,   799., ..., -2587.,   791., -1725.],
        ...,
...
        ...,
        [ -721.,  -609.,  -401., ...,  -994.,  1545.,   -20.],
        [ -927.,  -597.,  -400., ...,  -969.,  1621.,  -378.],
        [-1026.,  -507.,  -546., ..., -1120.,  1560.,  -797.]],

       [[ -326.,    51.,    85., ...,  -493.,  2676.,   235.],
        [ -226.,    66.,  -341., ...,  -557.,  2523.,   352.],
        [  -49.,   -37.,  -458., ...,  -682.,  2541.,   407.],
        ...,
        [ -643.,  -789.,  -497., ..., -1049.,  1737.,   286.],
        [ -982.,  -789.,  -527., ..., -1175.,  1695.,  -122.],
        [-1154.,  -763.,  -401., ..., -1053.,  1524.,  -548.]],

       [[ -193.,    75.,    63., ...,  -302.,  2612.,   407.],
        [ -126.,   -32.,   -94., ...,  -211.,  2841.,   755.],
        [   30.,  -150.,  -370., ...,  -248.,  3009.,   734.],
        ...,
        [ -478., -1041.,  -556., ...,  -986.,  1473.,   295.],
        [ -794.,  -963.,  -701., ..., -1193.,  1365.,   -20.],
        [ -995.,  -885.,  -689., ..., -1056.,  1330.,  -265.]]])
Coordinates:
  * latitude     (latitude) float64 224B -15.62 -15.88 -16.12 ... -22.12 -22.38
  * longitude    (longitude) float64 264B 25.12 25.38 25.62 ... 32.88 33.12
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 328B 1981-01-01 1982-01-01 ... 2021-01-01
Attributes:
    nodata:                 nan
    spi_calibration_begin:  1981-01-01 00:00:00
    spi_calibration_end:    2018-01-01 00:00:00

**Run bias correction**

In [36]:
# Bias correction
index_bc = run_bias_correction(
    anomaly_fc,
    anomaly_obs,
    params.end_season,
    params.monitoring_year,
    int(params.issue),
    nearest_neighbours=8,
    enso=True,
)
display(index_bc)

<xarray.DataArray 'scen' (longitude: 33, latitude: 28, time: 51)> Size: 377kB
array([[[  354.66885246,   435.75737705,  -136.96909091, ...,
          -169.48454545,   685.95763747,   656.83462322],
        [  439.82068966,   346.98275862,   -57.97621145, ...,
          -229.01321586,   788.52524462,   741.35368558],
        [  520.43092784,    76.69032258,    39.99677419, ...,
          -299.02352941,   892.52880734,   837.22752294],
        ...,
        [  677.22105263,  -725.92047619,   816.28459658, ...,
          -613.68712871,    -6.13432836,   791.90122249],
        [  680.99735099,  -753.95826772,   826.71637717, ...,
          -543.42394958,   -20.00138408,   773.45384615],
        [  686.63692661,  -789.47122905,   781.45642202, ...,
          -385.37620397,    34.25524476,   698.95114679]],

       [[  486.08541667,   439.38958333,  -127.64340426, ...,
          -133.13787234,   810.59145183,   679.2568521 ],
        [  486.08859649,   329.96578947,   -42.75798319, ...,
          -191.70588235,   841.67824377,   693.16055046],
        [  553.72608696,   119.55555556,    88.12941176, ...,
          -260.11414634,   946.01718464,   815.32906764],
...
        [ -731.82406015, -1024.21603053,  -211.51578947, ...,
           224.36890459,  -374.072103  ,   723.74900222],
        [ -783.92976589, -1192.80038388,  -303.23809524, ...,
          -199.36190476,  -416.0462585 ,   790.8644694 ],
        [ -820.22264151, -1297.26353791,  -345.19428571, ...,
          -380.78737542,  -415.52159468,   817.98500189]],

       [[  424.62779661,  1368.70601266,   723.78847458, ...,
           835.05710227,   330.68315018,    59.78388278],
        [  482.30527704,  1299.56897507,   700.02875989, ...,
           881.12048611,   296.88292683,   146.35609756],
        [  401.8765625 ,  1231.66057692,   707.371875  , ...,
           895.95172414,    29.79863014,    32.51130137],
        ...,
        [ -910.39213974, -1150.38636364,  -380.5640625 , ...,
            87.78736059,  -466.4171875 ,   657.18028169],
        [ -812.27727273, -1230.96462094,  -412.31103679, ...,
          -275.99313725,  -429.64882943,   762.09313456],
        [ -810.14725275, -1323.96727502,  -438.69452055, ...,
          -428.46027397,  -409.97260274,   806.88354839]]])
Coordinates:
  * longitude    (longitude) float64 264B 25.12 25.38 25.62 ... 32.88 33.12
  * latitude     (latitude) float64 224B -15.62 -15.88 -16.12 ... -22.12 -22.38
  * time         (time) datetime64[ns] 408B 2023-01-01 2023-01-02 ... 2023-02-20
    spatial_ref  int32 4B 4326
Attributes:
    nodata:                 nan
    units:                  
    spi_calibration_begin:  1981-01-01 00:00:00
    spi_calibration_end:    2018-02-20 00:00:00
    history:                [2024-07-22 11:07:48] : Bias-adjusted with Empiri...
    bias_adjustment:        EmpiricalQuantileMapping(group=Grouper(name='time...

**Run probabilities**

In [48]:
# Change dryspell sign as we compare values to a negative threshold to get probabilities
if params.index == "dryspell":
    anomaly_fc *= -1
    index_bc *= -1
    anomaly_obs *= -1

In [38]:
# Probabilities without Bias Correction
probabilities = compute_probabilities(
    anomaly_fc.where(anomaly_fc.time.dt.year == params.monitoring_year, drop=True),
    levels=params.intensity_thresholds,
).round(2)
display(probabilities)

<xarray.DataArray 'tp' (category: 2, latitude: 28, longitude: 33)> Size: 15kB
array([[[0.24, 0.22, 0.24, ..., 0.22, 0.22, 0.22],
        [0.24, 0.22, 0.24, ..., 0.24, 0.24, 0.24],
        [0.24, 0.2 , 0.22, ..., 0.27, 0.25, 0.24],
        ...,
        [0.24, 0.24, 0.24, ..., 0.31, 0.31, 0.35],
        [0.22, 0.22, 0.22, ..., 0.31, 0.31, 0.31],
        [0.2 , 0.22, 0.2 , ..., 0.31, 0.29, 0.31]],

       [[0.39, 0.37, 0.37, ..., 0.41, 0.39, 0.35],
        [0.35, 0.39, 0.35, ..., 0.45, 0.39, 0.39],
        [0.33, 0.33, 0.33, ..., 0.43, 0.41, 0.39],
        ...,
        [0.29, 0.27, 0.29, ..., 0.45, 0.45, 0.39],
        [0.33, 0.29, 0.29, ..., 0.45, 0.43, 0.41],
        [0.33, 0.31, 0.31, ..., 0.45, 0.39, 0.41]]])
Coordinates:
  * latitude     (latitude) float64 224B -15.62 -15.88 -16.12 ... -22.12 -22.38
  * longitude    (longitude) float64 264B 25.12 25.38 25.62 ... 32.88 33.12
    spatial_ref  int32 4B 4326
  * category     (category) object 16B 'Moderate' 'Normal'

In [39]:
# Probabilities after Bias Correction
probabilities_bc = compute_probabilities(
    index_bc, levels=params.intensity_thresholds
).round(2)
display(probabilities_bc)

<xarray.DataArray 'scen' (category: 2, longitude: 33, latitude: 28)> Size: 15kB
array([[[0.22, 0.25, 0.29, ..., 0.04, 0.06, 0.08],
        [0.22, 0.22, 0.27, ..., 0.06, 0.08, 0.1 ],
        [0.24, 0.24, 0.27, ..., 0.12, 0.12, 0.12],
        ...,
        [0.33, 0.29, 0.29, ..., 0.31, 0.31, 0.35],
        [0.29, 0.35, 0.33, ..., 0.31, 0.31, 0.33],
        [0.35, 0.37, 0.35, ..., 0.35, 0.31, 0.31]],

       [[0.39, 0.41, 0.41, ..., 0.27, 0.27, 0.25],
        [0.43, 0.43, 0.45, ..., 0.27, 0.25, 0.24],
        [0.43, 0.43, 0.45, ..., 0.27, 0.25, 0.24],
        ...,
        [0.45, 0.47, 0.49, ..., 0.51, 0.53, 0.51],
        [0.47, 0.45, 0.51, ..., 0.51, 0.51, 0.49],
        [0.45, 0.43, 0.51, ..., 0.53, 0.51, 0.51]]])
Coordinates:
  * longitude    (longitude) float64 264B 25.12 25.38 25.62 ... 32.88 33.12
  * latitude     (latitude) float64 224B -15.62 -15.88 -16.12 ... -22.12 -22.38
    spatial_ref  int32 4B 4326
  * category     (category) object 16B 'Moderate' 'Normal'

**Admin-2 level aggregation**

In [40]:
# Aggregate by district
probs_district = aggregate_by_district(probabilities, gdf, params)
probs_bc_district = aggregate_by_district(probabilities_bc, gdf, params)

# Build single xarray with merged unbiased/biased probabilities
probs_by_district = merge_un_biased_probs(
    probs_district, probs_bc_district, params, period_name
)
display(probs_by_district)

<xarray.Dataset> Size: 2kB
Dimensions:      (district: 65, category: 2, issue: 1, index: 1)
Coordinates:
  * district     (district) object 520B 'Beitbridge' 'Bikita' ... 'Zvishavane'
  * category     (category) object 16B 'Moderate' 'Normal'
  * issue        (issue) int64 8B 10
  * index        (index) object 8B 'spi ND'
    spatial_ref  int32 4B 0
Data variables:
    prob         (index, district, category, issue) float64 1kB 0.2978 ... 0.45

**Dataframe formatting**

In [41]:
# Merge probabilities with triggers
probs_df, merged_df = merge_probabilities_triggers_dashboard(
    probs_by_district, triggers_df, params, period_name
)

In [42]:
probs_df

,district,category,issue,index,prob,aggregation
0,Beitbridge,Moderate,10,SPI ND,0.30,SPI 2
1,Beitbridge,Normal,10,SPI ND,0.41,SPI 2
2,Bikita,Moderate,10,SPI ND,0.31,SPI 2
3,Bikita,Normal,10,SPI ND,0.43,SPI 2
4,Bindura,Moderate,10,SPI ND,0.33,SPI 2
...,...,...,...,...,...,...
125,Zaka,Normal,10,SPI ND,0.42,SPI 2
126,Zvimba,Moderate,10,SPI ND,0.31,SPI 2
127,Zvimba,Normal,10,SPI ND,0.42,SPI 2
128,Zvishavane,Moderate,10,SPI ND,0.32,SPI 2


In [45]:
merged_df

,district,index,category,window,issue_ready,issue_set,trigger_ready,trigger_set,vulnerability,prob_ready,prob_set
0,Beitbridge,DRYSPELL JF,Normal,Window 2,7.0,8.0,0.14,0.30,NRT,NaN,NaN
1,Beitbridge,SPI DJF,Normal,Window 2,7.0,8.0,0.38,0.00,NRT,NaN,NaN
2,Beitbridge,SPI FM,Normal,Window 2,9.0,10.0,0.31,0.35,NRT,NaN,NaN
3,Beitbridge,SPI JFM,Normal,Window 2,10.0,11.0,0.00,0.37,NRT,NaN,NaN
4,Beitbridge,SPI ND,Normal,Window 1,6.0,7.0,0.35,0.11,NRT,0.24,NaN
...,...,...,...,...,...,...,...,...,...,...,...
73,Rushinga,SPI DJ,Normal,Window 2,9.0,10.0,0.23,0.37,NRT,NaN,NaN
74,Rushinga,SPI FM,Normal,Window 2,11.0,12.0,0.34,0.27,NRT,NaN,NaN
75,Rushinga,SPI JF,Normal,Window 2,8.0,9.0,0.08,0.34,NRT,NaN,NaN
76,Rushinga,SPI ND,Normal,Window 1,6.0,7.0,0.30,0.26,NRT,0.29,NaN
